In [4]:
from path import Path

import torch
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
import models
from tqdm import tqdm

import torchvision.transforms as transforms
import flow_transforms
from imageio.v2 import imread, imwrite
import numpy as np
from util import flow2rgb


# myimport 
import matplotlib.pyplot as plt

In [21]:
path_to_model = "./pretrained_model/flownets_EPE1.951.pth.tar"

#Load model

def set_cuda():
    # setting the cuda device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)
    return device


def load_model(model_path, cuda_device):
    # Importing the model
    network_data = torch.load(model_path, map_location=cuda_device)
    print("=> using pre-trained model '{}'".format(network_data['arch']))
    model = models.__dict__[network_data['arch']](network_data).to(cuda_device)
    model.eval()
    cudnn.benchmark = True

    if 'div_flow' in network_data.keys():
        div_flow = network_data['div_flow']
    else:
        div_flow = 20.0

    return model, div_flow




@torch.no_grad()
def inference_direct(nb_start, nb_end, name_of_the_figure, cuda_device, model, div_flow):
    # create the folder for saving the results
    path_to_save = Path('my_results')
    if not path_to_save.exists():
        path_to_save.mkdir()


    # Data loading code
    input_transform = transforms.Compose([
        flow_transforms.ArrayToTensor(),
        transforms.Normalize(mean=[0,0,0], std=[255,255,255]),
        transforms.Normalize(mean=[0.411,0.432,0.45], std=[1,1,1])
    ])

    ###############importing images and nework form the paths################
    path_to_sequesnces = "./sequences-train/"
    name = name_of_the_figure + '-'

    
    ###############importing images and nework form the paths################

    n_img_1 = 1
    path_img1 = path_to_sequesnces + name + str(n_img_1).zfill(3) + ".bmp"
    img1 = imread(path_img1)
    img1 = input_transform(img1)

    for i in range(nb_start, nb_end):

        n_img_2 = i+1
        path_img2 = path_to_sequesnces + name + str(n_img_2).zfill(3) + ".bmp"
        img2 = imread(path_img2)
        img2 = input_transform(img2)

        input_var = torch.cat([img1, img2]).unsqueeze(0)
        input_var = input_var.to(cuda_device)
        output = model(input_var)
        output = F.interpolate(output, size=img1.size()[-2:], mode = "bilinear", align_corners=False)
        
        flow_output = output.squeeze(0)

        file_name = "./my_results/" + name + str(n_img_1).zfill(3) + "-" + str(n_img_2).zfill(3)
        rgb_flow = flow2rgb(div_flow*flow_output, max_value=None)
        to_save_rgb = (rgb_flow * 255).astype(np.uint8).transpose(1,2,0)
        imwrite(file_name + '.png', to_save_rgb)

        to_save_np = (div_flow*flow_output).cpu().numpy().transpose(1,2,0)
        np.save(file_name + '.npy', to_save_np)

    print("End of flow calculation for direct integration")

def propagate_mask(flow, img_current, mask_begin):
    new_mask = np.zeros(shape=img_current.shape[:2], dtype=np.uint8)
    for x in range(img_current.shape[0]):
        for y in range(img_current.shape[1]):
            x_, y_ = np.rint(x+flow[x,y,1]).astype(int), np.rint(y+flow[x,y,0]).astype(int)
            if (x_>=0) and (x_<img_current.shape[0]) and (y_>=0) and (y_<img_current.shape[1]):
                if mask_begin[x_,y_] > 0:
                    new_mask[x,y] = 255
    return new_mask

def propagate_mask_direct(nb_start, nb_end, name_in_the_figure):
    # generate all the propagated masks for the direct method

    original_mask = imread(f"./sequences-train/{name_in_the_figure}-001.png")
    first_img = imread(f'./sequences-train/{name_in_the_figure}-001.bmp')
    imwrite( f"./my_results/{name_in_the_figure}-mask_pro-001-001.png", original_mask) # For coherence when reading them later


    for i in range(nb_start+1, nb_end+1):
        flow = np.load( f"./my_results/{name_in_the_figure}-001-" + str(i).zfill(3) + '.npy')    
        current_mask = propagate_mask(flow, img_current= first_img, mask_begin = original_mask)

        imwrite( f"./my_results/{name_in_the_figure}-mask_pro-001-"+ str(i).zfill(3) +'.png', current_mask)

    print("End of mask propagation for direct integration")



In [22]:
nb_start = 1
nb_end   = 50

device = set_cuda()
model, div_flow = load_model(path_to_model, device)
inference_direct(nb_start,nb_end, "swan", device, model, div_flow)

propagate_mask_direct(nb_start, nb_end, "swan")


cpu
=> using pre-trained model 'flownets'
End of flow calculation for direct integration
End of mask propagation for direct integration
